### Q1(f) - Cochrane and Campbell - Consumptions with Habits

First we need to derive expressions for the risk free rate and equity returns. We are given 

\begin{align*}
u(c_t) = \frac{\Big(\frac{c_t}{\bar c_{t-1}}\Big)^\gamma - 1}{1-\gamma}
\end{align*}

Thus,

\begin{align*}
u'(c_t) = \frac{\gamma}{1-\gamma}c_t^{\gamma-1}\bar c_{t-1}^{-\gamma}.
\end{align*}

Therefore,
\begin{align*}
\frac{u'(c_{t+1})}{u'(c_t)} &= \frac{c_{t+1}^{\gamma-1}\bar c_{t}^{-\gamma}}{c_t^{\gamma-1}\bar c_{t-1}^{-\gamma}}\\
&= \Big(\frac{c_{t+1}}{c_t}\Big)^{\gamma-1}\Big(\frac{\bar c_{t}}{\bar c_{t-1}}\Big)^{-\gamma}\\
&= \lambda_{t+1}^{\gamma-1} \lambda_t^{-\gamma}
\end{align*}

Now, the price of the risk free asset is given by
\begin{align*}
p_t &= E_t\Big[\beta \frac{u'(c_{t+1})}{u'(c_t)} \Big]\\
&= \beta \lambda_t^{-\gamma} E_t [\lambda_{t+1}^{\gamma-1}]\\
\implies R_t^{-1} &= \beta \lambda_t^{-\gamma} E_t [\lambda_{t+1}^{\gamma-1}]
\end{align*}

As previously, we can drop the time subscripts because the equilibrium values in this economy are time-invariant functions of the state. So, we can write the inverse of the risk free rates in each state as a $2 \times 1$ vector:
\begin{align*}
R^{-1} = \beta \Lambda_1 P \Lambda_2,
\end{align*}
where
\begin{align*}
\Lambda_1 = 
\begin{bmatrix}
 \lambda_1^{-\gamma} & 0\\
 0 & \lambda_2^{-\gamma}
\end{bmatrix}
\text{ and }\Lambda_2 = 
\begin{bmatrix}
 \lambda_1^{\gamma-1} & 0\\
 0 & \lambda_2^{\gamma-1}
\end{bmatrix}.
\end{align*}
We calculate the unconditional risk free rate as before.

Next, we want to price equity. We start with the general formula:
\begin{align*}
p_t = \beta E_t \Big[\frac{u'(c_{t+1})}{u'(c_t)} (y_{t+1}+ p_{t+1}) \Big]
\end{align*}

After doing some algebra, we get the following expression for the price/dividend ratio:
\begin{align*}
w_t = \beta \lambda_t^{-\gamma} E_t [\lambda_{t+1}^{\gamma}(1+w_{t+1})]
\end{align*}
Again, we can drop the time subscripts, write this in matrix form and solve for $w$ as we do in the canconical model. We end up with:
\begin{align*}
w = \beta (I - \beta \Lambda_1 P \Lambda_3)^{-1} \Lambda_1 P \Lambda_3 i,
\end{align*}
where
\begin{align*}
\Lambda_3 = 
\begin{bmatrix}
 \lambda_1^{\gamma} & 0\\
 0 & \lambda_2^{\gamma}
\end{bmatrix},
\end{align*}
and $i$ is a $2\times 1$ vector of 1's.

Now we can solve for equity returns in each state and derive the unconditional equity return just as we did previously. I implement the model below using the parametrization from part (c).

In [2]:
function habits(gamma,beta) 

    # States
    s1 = 1.02252;
    s2 = 1-(6.785/100);

    # Lambda matrix
    Lambda1 = [s1^(-gamma) 0; 0 s2^(-gamma)];
    Lambda2 = [s1^(gamma) 0; 0 s2^(gamma)];

    #Lambda column vector
    lambdavec = [s1, s2];

    # Vector and matrix of ones
    i=ones(2);
    I=eye(2);

    # Transition matrix
    p=0.987
    q=0.516
    P = [p 1-p; 1-q q];

    # Ergodic distribution across states
    Pi = P^1000;
    Pivec = Pi[1,:];
    
    # Risk free rate vector
    Rinv = beta*(Lambda1)*(P*(lambdavec.^(gamma-1)))
    R = Rinv.^(-1)

    #Unconditional (average) risk free rate
    Rbar = dot(Pivec,R)

    #Price/dividend ratios
    w=beta*(inv(I-beta*Lambda1*P*Lambda2))*Lambda1*P*Lambda2*i

    # Expected equity returns
    ERS = P*((lambdavec).*(1+w)).*[1/w[1] 1/w[2]]'

    # Unconditional (average) equity return
    RSbar = dot(Pivec,ERS)
    
    # Equity premium, per cent
    ep = (RSbar - Rbar)*100
    
    # RESULTS VECTOR
    ANS = [gamma, Rbar, RSbar, ep]
    
    return ANS
end;

### Results

In [3]:
Results=hcat(["gamma", "Rf", "Re", "Re-Rf"],habits(2,0.99),habits(4,0.99),habits(6,0.99),habits(10,0.99))

4×5 Array{Any,2}:
 "gamma"  2.0        4.0      6.0      10.0    
 "Rf"     1.03051    1.03034  1.02987   1.0284 
 "Re"     1.03067    1.03172  1.03365   1.04047
 "Re-Rf"  0.0154977  0.13822  0.37842   1.2076 